<img src='./img/logo.jpg' width=800 align='left'>

_Autor: Evandro Avellar_

# Dia 01 - Importar Dados

Dados de empréstimos dos acervos do sistema de bibliotecas da UFRN. 

In [3]:
import pandas as pd
import os

## Coleta e Organização dos Dados

## Unificar em um único Dataframe todos os dados pertinentes para a análise. 

### Dados de empréstimos

* Listando os arquivos csv de Empréstimos para unificá-los num único DataFrame.

> https://docs.python.org/pt-br/3/library/os.html#os.listdir

In [83]:
arquivos = os.listdir('./Dia_1-Importando_dados/Datasets/dados_emprestimos/')
arquivos

['emprestimos-20101.csv',
 'emprestimos-20102.csv',
 'emprestimos-20111.csv',
 'emprestimos-20112.csv',
 'emprestimos-20121.csv',
 'emprestimos-20122.csv',
 'emprestimos-20131.csv',
 'emprestimos-20132.csv',
 'emprestimos-20141.csv',
 'emprestimos-20142.csv',
 'emprestimos-20151.csv',
 'emprestimos-20152.csv',
 'emprestimos-20161.csv',
 'emprestimos-20162.csv',
 'emprestimos-20171.csv',
 'emprestimos-20172.csv',
 'emprestimos-20181.csv',
 'emprestimos-20182.csv',
 'emprestimos-20191.csv',
 'emprestimos-20192.csv',
 'emprestimos-20201.csv']

Após listar os arquivos a serem importados, eles serão unificados em um único DataFrame através do método ```pd.concat()``` do Pandas.

> https://pandas.pydata.org/docs/reference/api/pandas.concat.html

In [84]:
tabela_x = pd.DataFrame()

for arquivo in arquivos:
    tabela_1 = pd.read_csv(f'./Dia_1-Importando_dados/Datasets/dados_emprestimos/{arquivo}')
    tabela_x = pd.concat([tabela_x, tabela_1])    

In [85]:
tabela_x.reset_index(inplace=True)

### Dados do acervo

Para utilização do Pandas na leitura de arquivo .parquet é necessário instalar a biblioteca ___pyarrow___ e o método ___pd.read_parquet()___

https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html

In [24]:
!pip install pyarrow

     -------------------------------------- 21.5/21.5 MB 588.6 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [86]:
acervo = pd.read_parquet('./Dia_1-Importando_dados/Datasets/dados_exemplares.parquet', engine='pyarrow')
acervo

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15
...,...,...,...,...,...,...,...
568580,1353472,2021011150,Acervo Circulante,Biblioteca Setorial do Núcleo de Educação da I...,REGULAR,951,268231
568581,1353473,2019013454,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,997,268233
568582,1353474,2019012811,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,987,268234


## Identificando dados nulos e vazios

https://pandas.pydata.org/docs/reference/api/pandas.isna.html

https://pandas.pydata.org/docs/reference/api/pandas.isnull.html

In [87]:
# isna
print(tabela_x.isna().sum())

index                         0
id_emprestimo                 0
codigo_barras                 0
data_renovacao          1285720
data_emprestimo               0
data_devolucao             6471
matricula_ou_siape         3170
tipo_vinculo_usuario          0
dtype: int64


In [88]:
# isnull
print(tabela_x.isnull().sum())

index                         0
id_emprestimo                 0
codigo_barras                 0
data_renovacao          1285720
data_emprestimo               0
data_devolucao             6471
matricula_ou_siape         3170
tipo_vinculo_usuario          0
dtype: int64


In [89]:
# isna
print(acervo.isna().sum())

id_exemplar         0
codigo_barras       0
colecao             0
biblioteca          0
status_material     0
localizacao         0
registro_sistema    0
dtype: int64


In [90]:
# isnull
print(acervo.isnull().sum())

id_exemplar         0
codigo_barras       0
colecao             0
biblioteca          0
status_material     0
localizacao         0
registro_sistema    0
dtype: int64


As colunas que apresentam dados nulos não afetarão a junção dos dataframes nesse momento.

## Removendo dados duplicados

Não poderemos trabalhar com registros com dados do código de barras duplicados no acervo pois serão utilizados como chaves para o merge.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html

In [91]:
print(acervo['codigo_barras'].nunique())
print(len(acervo))

545049
546237


Para tabela dos empréstimos não poderão ter registros duplicados com mesmo número na coluna id_emprestimo.

In [92]:
print(tabela_x['id_emprestimo'].nunique())
print(len(tabela_x))

2139267
2258018


Eliminando os registros duplicados com o método ___drop_duplicates___

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

In [93]:
acervo_df = acervo.drop_duplicates(subset='codigo_barras', keep='first')
emprestimos_df = tabela_x.drop_duplicates(subset='id_emprestimo', keep='first')

In [94]:
print(emprestimos_df['id_emprestimo'].nunique())
print(len(emprestimos_df))

2139267
2139267


In [95]:
print(acervo_df['codigo_barras'].nunique())
print(len(acervo_df))

545049
545049


### Merge entre dos dois DataFrames (Acervo e Empréstimos)

Mesclando com os dados do acervo, para entender, por exemplo, de qual biblioteca era o material emprestado ou a qual tema ele se referia. A relação se dá pela coluna de código de barras de cada material. 

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [96]:
df_1 = pd.merge(emprestimos_df, acervo_df, on = "codigo_barras")

In [97]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955743 entries, 0 to 1955742
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   index                 int64  
 1   id_emprestimo         int64  
 2   codigo_barras         object 
 3   data_renovacao        object 
 4   data_emprestimo       object 
 5   data_devolucao        object 
 6   matricula_ou_siape    float64
 7   tipo_vinculo_usuario  object 
 8   id_exemplar           int64  
 9   colecao               object 
 10  biblioteca            object 
 11  status_material       object 
 12  localizacao           int64  
 13  registro_sistema      int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 223.8+ MB


In [98]:
# isna
print(df_1.isna().sum())

index                         0
id_emprestimo                 0
codigo_barras                 0
data_renovacao          1096082
data_emprestimo               0
data_devolucao             6138
matricula_ou_siape         2705
tipo_vinculo_usuario          0
id_exemplar                   0
colecao                       0
biblioteca                    0
status_material               0
localizacao                   0
registro_sistema              0
dtype: int64


In [99]:
# isnull
print(df_1.isnull().sum())

index                         0
id_emprestimo                 0
codigo_barras                 0
data_renovacao          1096082
data_emprestimo               0
data_devolucao             6138
matricula_ou_siape         2705
tipo_vinculo_usuario          0
id_exemplar                   0
colecao                       0
biblioteca                    0
status_material               0
localizacao                   0
registro_sistema              0
dtype: int64


DataFrame preparado para iniciar as análises

## Limpeza de Dados

### Criação de coluna de classes do CDU

"Os itens do acervo em uma biblioteca são organizados por um sistema de classificação de acordo com o respectivo tema. Existem diversos sistemas, mas este conjunto está de acordo com a CDU - Classificação Decimal Universal. Esta classificação é decimal, pois varia de acordo com a classe de cada assunto:

*	000 a 099: Generalidades. Ciência e conhecimento.
*	100 a 199: Filosofia e psicologia.
*	200 a 299: Religião.
*	300 a 399: Ciências sociais.
*	400 a 499: Classe vaga. Provisoriamente não ocupada.
*	500 a 599: Matemática e ciências naturais.
*	600 a 699: Ciências aplicadas.
*	700 a 799: Belas artes.
*	800 a 899: Linguagem. Língua. Linguística.
*	900 a 999: Geografia. Biografia. História."


Portanto, se um material tiver um código de localização 720, ele está dentro da classe geral de “Belas Artes”; ou se tiver um código 028, estará dentro da classe geral de “Generalidades. Ciência e conhecimento”.

Para isso, crie uma nova coluna com os valores da localização, para refletir a respectiva classe geral na CDU.

In [64]:
def categorizar(num):
    if num < 100:
        return 'Generalidades. Ciência e conhecimento'
    elif num >= 100 and num < 199: 
        return 'Filosofia e psicologia'
    elif num >= 200 and num < 299:
        return 'Religião'
    elif num >= 300 and num <= 399:
        return 'Ciências sociais'
    elif num >= 400 and num <= 499: 
        return 'Classe vaga. Provisoriamente não ocupada'
    elif num >= 500 and num <= 599: 
        return 'Matemática e ciências naturais'
    elif num >= 600 and num <= 699: 
        return 'Ciências aplicadas.'
    elif num >= 700 and num <= 799: 
        return 'Belas artes'
    elif num >= 800 and num <= 899: 
        return 'Linguagem. Língua. Linguística'
    elif num >= 900 and num <= 999: 
        return 'Geografia. Biografia. História'
    else:
        return "Erro"

In [100]:
df_1['Classe'] = df_1['localizacao'].apply(categorizar)

### Exclusão da coluna "registro_sistema" 

A coluna "registro_sistema" não está fazendo sentido para essa análise, por isso será excluída.

In [101]:
df_1 = df_1[['index', 'id_emprestimo', 'codigo_barras', 'data_renovacao','data_emprestimo', 'data_devolucao', 'matricula_ou_siape',
       'tipo_vinculo_usuario', 'id_exemplar', 'colecao', 'biblioteca','status_material', 'localizacao', 'Classe']]

In [102]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955743 entries, 0 to 1955742
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   index                 int64  
 1   id_emprestimo         int64  
 2   codigo_barras         object 
 3   data_renovacao        object 
 4   data_emprestimo       object 
 5   data_devolucao        object 
 6   matricula_ou_siape    float64
 7   tipo_vinculo_usuario  object 
 8   id_exemplar           int64  
 9   colecao               object 
 10  biblioteca            object 
 11  status_material       object 
 12  localizacao           int64  
 13  Classe                object 
dtypes: float64(1), int64(4), object(9)
memory usage: 223.8+ MB


### Transformar a coluna "matricula_ou_siape" no formato String

A coluna da matricula (“matricula_ou_siape”) não está com um formato muito legível. 
Será transformada em formato String. 

In [103]:
df_1['matricula_ou_siape'] = df_1['matricula_ou_siape'].apply(str)

In [104]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955743 entries, 0 to 1955742
Data columns (total 14 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   index                 int64 
 1   id_emprestimo         int64 
 2   codigo_barras         object
 3   data_renovacao        object
 4   data_emprestimo       object
 5   data_devolucao        object
 6   matricula_ou_siape    object
 7   tipo_vinculo_usuario  object
 8   id_exemplar           int64 
 9   colecao               object
 10  biblioteca            object
 11  status_material       object
 12  localizacao           int64 
 13  Classe                object
dtypes: int64(4), object(10)
memory usage: 223.8+ MB


In [105]:
df_1

,index,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,Classe
0,0,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2008023265.0,ALUNO DE GRADUAÇÃO,13259,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630,Ciências aplicadas.
1,17278,726965,L095049,2010/03/31 12:41:08.684000000,2010/03/01 12:40:56.212000000,2010/04/30 14:42:35.366000000,2009121584.0,ALUNO DE PÓS-GRADUAÇÃO,13259,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630,Ciências aplicadas.
2,1,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2008023265.0,ALUNO DE GRADUAÇÃO,70865,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647,Ciências aplicadas.
3,8090,717763,L167050,NaN,2010/02/11 11:16:30.335000000,2010/03/04 08:54:32.262000000,2008050352.0,ALUNO DE GRADUAÇÃO,70865,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647,Ciências aplicadas.
4,20305,729996,L167050,2010/03/17 09:08:26.874000000,2010/03/04 13:36:26.599000000,2010/03/31 18:04:29.669000000,200722565.0,ALUNO DE GRADUAÇÃO,70865,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647,Ciências aplicadas.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955738,26488,2989018,2009036492,NaN,2020/03/17 19:49:10.187000000,2020/12/03 08:12:20.000000000,20150109837.0,ALUNO DE GRADUAÇÃO,274820,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,676,Ciências aplicadas.
1955739,26490,2989020,2014062640,NaN,2020/03/17 19:51:15.700000000,2021/01/07 11:16:28.000000000,20190035012.0,ALUNO DE GRADUAÇÃO,1157915,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,228,Religião
1955740,26502,2989032,2018003324,NaN,2020/03/17 20:00:11.416000000,2021/08/30 13:29:44.000000000,20191021693.0,ALUNO DE PÓS-GRADUAÇÃO,1317173,Acervo Circulante,Biblioteca Setorial da Faculdade de Ciências d...,REGULAR,784,Belas artes
1955741,26519,2989049,2006024313,NaN,2020/03/17 20:47:18.741000000,NaN,20150129867.0,ALUNO DE GRADUAÇÃO,206073,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,663,Ciências aplicadas.
